In [1]:
#load libraries

import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from collections import defaultdict
import csv
import itertools
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math

In [2]:
# get soup of AMFI Website

url = 'https://www.amfiindia.com/nav-history-download'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')



# all Mutual Fund Names

NavDownMFName = []
for op in soup.find_all('select', {'class' : 'select', 'id' : 'NavDownMFName'})[0]:
    str_op = str(op)[15:].split('">')
    if len(str_op) > 1 and str_op[0] != '':
        MF_Value = str_op[0] 
        MF_Name = str_op[1].split('<')[0]
        NavDownMFName.append(MF_Value +' ' + MF_Name)
        
        

# types of scheme
        
NavDownType = []
for op in soup.find_all('select', {'class' : 'select', 'id' : 'NavDownType'})[0]:
    str_op = str(op)[15:].split('">')
    if len(str_op) > 1 and str_op[0] != '':
        NDT_Value = str_op[0] 
        NDT_Name = str_op[1].split('<')[0]
        print(NDT_Value, NDT_Name)
        NavDownType.append(NDT_Value +' ' + NDT_Name)



# Mutual Funds code list
        
code_lst = []
for name in NavDownMFName[1:]:
    code = name.split(" ")[0]
    code_lst.append(code)

-1 All
1 Open Ended Schemes
2 Close Ended Schemes
3 Interval Fund


In [3]:
# Mutual Funds name list

name_lst = []
for name in NavDownMFName[1:]:
    nam = name.split(" ", 1)[1]
    nam = nam.replace(" Mutual Fund", "")
    name_lst.append(nam)

pd.DataFrame(name_lst).to_csv("name.csv")

In [4]:
# Output dict

## {MF name : {mf plan : {Date : price}}}

final_dict = defaultdict(lambda:defaultdict(lambda: {}))

In [5]:
for code in code_lst[:71]:
    print(code)
    NDMFName = code                  ## Mutual Fund code
    NDType = '1'
    s_date = '01-01-2017' #%m-%d-%y    ## Starting date in str formate

    start_date = datetime.strptime(s_date, '%m-%d-%Y')    ## in datetime format
    end_date = start_date + relativedelta(months=1)       ## 1 month for monthly returns

    fields = ['scheme - plan']                            


    while end_date < datetime.now() :                     ## Get the data till the recent month


        ## dct = {MF name : {MF Scheme : { MF Plan : {Date : Price}}}}
        dct=defaultdict(lambda:defaultdict(lambda:defaultdict(lambda:{
                            }))) 

        e_date = end_date.strftime('%m-%d-%Y')            ## convert datetime to str formate
        month_year = start_date.strftime("%B-%Y")


        ## make url to get the data
        new_url = f'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?mf={NDMFName}&tp={NDType}&frmdt={s_date}&todt={e_date}'
        response = requests.get(new_url)

        ## Make list for each scheme
        schemes_list = response.text.split("Open Ended Schemes ")

        for scheme in schemes_list[1:]:
            lines = scheme.split('\r\n')               ## get every row
            mf_name = lines[3]                         ## Mutual fund name
            mf_scheme = lines[0][2:-2]                 ## Mutual fund scheme name
            mf_plan_lst = []                           ## MF plans under that schem like direct growth, regular growth etc.



            for line in lines[4:-2]:                   ## every day price data
                lst = line.split(';')

                if len(mf_plan_lst) < 1:               ## adding mf plans to mf_plan_lst
                    mf_plan_lst.append(lst[1])
                elif mf_plan_lst[-1] != lst[1]:        ## adding mf plans to lst if it is not in list
                    mf_plan_lst.append(lst[1])

                mf_plan = lst[1]
                dct[mf_name][mf_scheme][mf_plan][lst[-1]] = lst[-4]       ## adding values to dict


            lst_price_month = list(dct[mf_name][mf_scheme].values())      ## list of all the values and plans for that month  

            
            for i in range(len(lst_price_month)):
                ls = list(list(lst_price_month)[i].values())              ## price values for the specific plan for that monath

                try:
    #                 dif = float(ls[-1]) - float(ls[0])                    ## price diff for that month
    #                 dif_pct = round(dif/float(ls[0])*100, 3)
                    dif_pct = round(math.log(float(ls[-1])/float(ls[0]))*100,3)               ## % diff 
                    if dif_pct > 50 or dif_pct < -50:                     ## to remove outlier 
                        dif_pct = None           
                except:
                    dif_pct = None                                           
                final_dict[mf_name][mf_plan_lst[i]][month_year] = dif_pct





        start_date = end_date                                   ## Moving to next month
        end_date = end_date + relativedelta(months=1)

        ## converting in str formate
        s_date = start_date.strftime('%m-%d-%Y')
        e_date = end_date.strftime('%m-%d-%Y')

        fields.append(month_year)                               ## For header in csv

    # print(final_dict)

    try:                                                        ## checking if MF's data is available or not on AMFI website
        mf_name
    except:
        continue


        ## Making csv files for every Mutual Fund

    dw = final_dict[mf_name]
    with open(f"{mf_name}.csv", "w", encoding="utf-8") as f:
        w = csv.DictWriter(f, fields)
        w.writeheader()
        for k in dw:
            w.writerow({field: dw[k].get(field) or k for field in fields})

39
3
50
1
53
46
4
36
59
32
60
31
38
6
47
40
51
27
8
49
9
37
20
57
48
68
62
11
65
63
14
42
70
16
43
17
56
18
69
45
19
55
54
21
73
58
44
34
64
10
13
41
74
22
52
67
2
24
33
25
26
72
61
28
71
29
